## Import libraries

In [19]:
from fastapi import FastAPI, Request
from pydantic import BaseModel
import pandas as pd
from typing import List, Dict
import json
import os
import clickhouse_connect

import mlflow

import requests

from dotenv import load_dotenv
load_dotenv()

CH_USER = os.getenv('CH_USER')
CH_PASS = os.getenv('CH_PASS')
CH_IP = os.getenv('CH_IP')

from api.union_dfs import union_dfs
from api.df_preprocessor import df_preprocessor

root_path = "./api"
preprocessor_path = f"{root_path}/preprocessor"

client = clickhouse_connect.get_client(host=CH_IP, port=8123, username=CH_USER, password=CH_PASS)

your_mlflow_tracking_uri = f'{root_path}/mlruns' # for docker mlflow server
# your_mlflow_tracking_uri = "http://127.0.0.1:5000" # for local mlflow server
# your_mlflow_tracking_uri = MLFLOW_TRACKING_URI # for remote mlflow server
mlflow.set_tracking_uri(your_mlflow_tracking_uri)

## Spark Initialize

In [2]:
import findspark
findspark.init()

from pyspark import SparkContext, SparkConf, SQLContext

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import DataFrame as SparkDataFrame
from pyspark.sql import Window



# ml
from pyspark.ml import Pipeline as spk_pipeline
from pyspark.ml.feature import OneHotEncoder as spk_OneHotEncoder, StandardScaler as spk_StandardScaler, VectorAssembler as spk_VectorAssembler
from pyspark.ml.feature import MinMaxScaler as spk_MinMaxScaler, StringIndexer as spk_StringIndexer
from pyspark.ml.evaluation import RegressionEvaluator as spk_RegressionEvaluator

from pyspark.ml import Transformer
from pyspark.ml.param.shared import HasInputCol, HasOutputCol, Param






import os
#https://repo1.maven.org/maven2/com/github/housepower/clickhouse-native-jdbc/2.7.1/clickhouse-native-jdbc-2.7.1.jar
# spark connector https://github.com/ClickHouse/spark-clickhouse-connector
# https://mvnrepository.com/artifact/com.clickhouse
# https://github.com/housepower/ClickHouse-Native-JDBC, For Spark 3.2 and upper, Spark ClickHouse Connector (see upper) is recommended.
# https://clickhouse.com/docs/en/integrations/apache-spark/spark-native-connector
packages = [
    "com.clickhouse.spark:clickhouse-spark-runtime-3.5_2.12:0.8.0"
    # "com.github.housepower:clickhouse-spark-runtime-3.4_2.12:0.7.3"
    ,"com.clickhouse:clickhouse-jdbc:0.7.1-patch1"
    # ,"com.clickhouse:clickhouse-jdbc:0.6.0-patch5"
    ,"com.clickhouse:clickhouse-http-client:0.7.1-patch1"
    # ,"com.clickhouse:clickhouse-http-client:0.6.0-patch5"
    ,"org.apache.httpcomponents.client5:httpclient5:5.3.1"
    # for jdbc 2.7.1 required java 8/11
    # ,"com.github.housepower:clickhouse-native-jdbc:2.7.1"
    ,"ai.catboost:catboost-spark_3.5_2.12:1.2.7"
    ,"com.microsoft.azure:synapseml_2.12:1.0.8"

]

exclude_packages = [
    "org.scala-lang:scala-reflect"
    ,"org.apache.spark:spark-tags_2.12"
    ,"org.scalactic:scalactic_2.12"
    ,"org.scalatest:scalatest_2.12"
    ,"com.fasterxml.jackson.core:jackson-databind"
]



ram = 60
cpu = 22*3
# Define the application name and setup session
appName = "Connect To ClickHouse via PySpark"
spark = (SparkSession.builder
         .appName(appName)
         .config("spark.jars.packages", ",".join(packages))
         .config("spark.sql.catalog.clickhouse", "com.clickhouse.spark.ClickHouseCatalog")
         .config("spark.sql.catalog.clickhouse.host", CH_IP)
         .config("spark.sql.catalog.clickhouse.protocol", "http")
         .config("spark.sql.catalog.clickhouse.http_port", "8123")
         .config("spark.sql.catalog.clickhouse.user", CH_USER)
         .config("spark.sql.catalog.clickhouse.password", CH_PASS)
         .config("spark.sql.catalog.clickhouse.database", "default")
        #  .config("spark.spark.clickhouse.write.compression.codec", "lz4")
        #  .config("spark.clickhouse.read.compression.codec", "lz4")
        #  .config("spark.clickhouse.write.format", "arrow")
         #    .config("spark.clickhouse.write.distributed.convertLocal", "true") l
         #    .config("spark.clickhouse.write.repartitionNum", "1") 
         #.config("spark.clickhouse.write.maxRetry", "1000")
         #    .config("spark.clickhouse.write.repartitionStrictly", "true") 
         #    .config("spark.clickhouse.write.distributed.useClusterNodes", "false") 
        #  .config("spark.clickhouse.write.batchSize", "1000000")
         #.config("spark.sql.catalog.clickhouse.socket_timeout", "600000000")
        #  .config("spark.sql.catalog.clickhouse.connection_timeout", "600000000")
        #  .config("spark.sql.catalog.clickhouse.query_timeout", "600000000")
        #  .config("spark.clickhouse.options.socket_timeout", "600000000")
        #  .config("spark.clickhouse.options.connection_timeout", "600000000")
        #  .config("spark.clickhouse.options.query_timeout", "600000000")         
         .config("spark.executor.memory", f"{ram}g")
        #  .config("spark.executor.cores", "5")
         .config("spark.driver.maxResultSize", f"{ram}g")
         .config("spark.driver.memory", f"{ram}g")
         .config("spark.executor.memoryOverhead", f"{ram}g")
        #  .config("spark.sql.debug.maxToStringFields", "100000")
         .getOrCreate()
         )

# LightGBM set config https://microsoft.github.io/SynapseML/docs/Get%20Started/Install%20SynapseML/
# spark.conf.set("spark.jars.repositories", "https://mmlspark.azureedge.net/maven")
# spark.conf.set("spark.jars.excludes", ",".join(exclude_packages))
# spark.conf.set("spark.yarn.user.classpath.first", "true")
# spark.conf.set("spark.sql.parquet.enableVectorizedReader", "false")

#SedonaRegistrator.registerAll(spark)
# spark.conf.set("spark.sql.catalog.clickhouse", "xenon.clickhouse.ClickHouseCatalog")
# spark.conf.set("spark.sql.catalog.clickhouse.host", "127.0.0.1")
# spark.conf.set("spark.sql.catalog.clickhouse.protocol", "http")
# spark.conf.set("spark.sql.catalog.clickhouse.http_port", "8123")
# spark.conf.set("spark.sql.catalog.clickhouse.user", "default")
# spark.conf.set("spark.sql.catalog.clickhouse.password", "")
# spark.conf.set("spark.sql.catalog.clickhouse.database", "default")



from catboost_spark import CatBoostRegressor as CatBoostRegressor_spark
from synapse.ml.lightgbm import LightGBMRegressor as LightGBMRegressor_spark


spark.sql("use clickhouse")

:: loading settings :: url = jar:file:/opt/bitnami/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.clickhouse.spark#clickhouse-spark-runtime-3.5_2.12 added as a dependency
com.clickhouse#clickhouse-jdbc added as a dependency
com.clickhouse#clickhouse-http-client added as a dependency
org.apache.httpcomponents.client5#httpclient5 added as a dependency
ai.catboost#catboost-spark_3.5_2.12 added as a dependency
com.microsoft.azure#synapseml_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-88ed53cc-207c-4d60-88c0-342b30ae8eca;1.0
	confs: [default]
	found com.clickhouse.spark#clickhouse-spark-runtime-3.5_2.12;0.8.0 in central
	found com.clickhouse#clickhouse-jdbc;0.7.1-patch1 in central
	found com.clickhouse#clickhouse-client;0.7.1-patch1 in central
	found com.clickhouse#clickhouse-data;0.7.1-patch1 in central
	found com.clickhouse#clickhouse-http-client;0.7.1-patch1 in central
	found org.apache.httpcomponents.core5#httpcore5-h2;5.2 in central
	

DataFrame[]

## Server-Side: FastAPI Application

In [ ]:
app = FastAPI()

class DataFrameInput(BaseModel):
    data: List[Dict]  # Expecting a list of dictionaries as input

@app.post("/process-dataframe")
async def process_dataframe(input_data: DataFrameInput):
    # Convert the JSON data into a Pandas DataFrame
    df = pd.DataFrame(input_data.data)
    
    # Modify the DataFrame (example: add a new column)
    df["new_column"] = df["column1"] * 2  # Assuming 'column1' exists in the input
    
    # Convert the modified DataFrame back to JSON
    response_data = df.to_dict(orient="records")
    return {"data": response_data}






@app.get("/control")
async def get_control(id: int):
    control = client.query_df(f'''
        select * 
        from ycup.control yc
        where yc.id = {id}
        limit 10'''
    )


### clickhouse connect

In [11]:
client.query_df('SHOW TABLES IN ycup')

,name
0,control
1,localization
2,metadata


In [3]:
# Execute the query to retrieve the list of tables
tables = client.query_df('SHOW TABLES IN ycup')

# Execute the query
query = f'SELECT * FROM ycup.{tables.loc[0, 'name']} LIMIT 10'

# first query variant
result = client.query_df(query)

result


,id,stamp_ns,acceleration_level,steering
0,0,36479492,-929,5.739836
1,0,76459951,-926,5.280618
2,0,116678417,-918,5.039505
3,0,156788958,-908,4.734873
4,0,196857808,-897,4.387096
5,0,236974997,-892,4.014573
6,0,276890721,-892,3.627408
7,0,316915752,-901,3.191926
8,0,356884436,-911,2.748362
9,0,396895329,-918,2.286391


In [9]:
def make_df_id_test(control:pd.DataFrame, localization:pd.DataFrame, metadata:pd.DataFrame) -> pd.DataFrame:
    '''Make a model dataframe from control, localization and metadata dataframes for a single id'''

    def find_min_max(control:pd.DataFrame, localization:pd.DataFrame):
        '''Find min and max timestamp in localization for each timestamp in control dataframe'''
        control['loclz_stamp_ns_max'] = control['stamp_ns'].apply(lambda x: localization[localization['stamp_ns'] >= x]['stamp_ns'].min())
        control['loclz_stamp_ns_min'] = control['stamp_ns'].apply(lambda x: localization[localization['stamp_ns'] < x]['stamp_ns'].max())
        control['loclz_stamp_ns_min'] = control['loclz_stamp_ns_min'].apply(lambda x: localization['stamp_ns'].min() if pd.isnull(x) else x)
        control_2m = control.copy()
        control_2m.rename(columns={'stamp_ns':'ctrl_stamp_ns'}, inplace=True)
        return control_2m

    def merge_min_max(control_2m:pd.DataFrame, localization:pd.DataFrame):
        '''Merge min and max timestamp in localization for each timestamp in control dataframe'''
        control_3m = (control_2m.merge(localization, left_on='loclz_stamp_ns_max', right_on='stamp_ns', how='left', suffixes=('', '_max'))
                    .merge(localization, left_on='loclz_stamp_ns_min', right_on='stamp_ns', how='left', suffixes=('', '_min'))
        )
        control_3m.rename(columns={'x':'x_max'
                                ,'y':'y_max'
                                ,'z':'z_max'
                                ,'roll':'roll_max'
                                ,'pitch':'pitch_max'
                                ,'yaw':'yaw_max'
                                ,'stamp_ns':'stamp_ns_max'
                    }, inplace=True)
        control_3m.drop(columns=['loclz_stamp_ns_max', 'loclz_stamp_ns_min'], inplace=True)
        return control_3m

    def interpolate_coords(control_3m, col_min:str, col_max:str):
        '''Interpolate values between max and min values'''
    
        control_interpolated = (control_3m[['ctrl_stamp_ns', 'stamp_ns_max', 'stamp_ns_min', col_min, col_max]]
                .apply(lambda x:
                    x[col_min] if x['stamp_ns_max'] == x['stamp_ns_min']
                    else (x['ctrl_stamp_ns'] - x['stamp_ns_min']) / (x['stamp_ns_max'] - x['stamp_ns_min']) * (x[col_max] - x[col_min]) + x[col_min], axis=1
                    )
                )

        return control_interpolated


    def add_metadata(control:pd.DataFrame, metadata:pd.DataFrame):
        '''Add metada to each row in control dataframe'''
        # Make a copy to avoid SettingWithCopyWarning
        control_model = control.copy()
        for col in metadata.columns:
            control_model[col] = metadata.loc[0, col]  # Set the entire column in the copy
        
        return control_model

    # find min and max timestamp in localization for each timestamp in control dataframe
    control_2m = find_min_max(control, localization)
    # merge min and max timestamp in localization for each timestamp in control dataframe
    control_3m = merge_min_max(control_2m, localization)
    
    
    coords_cols = ['x', 'y', 'z', 'roll', 'pitch', 'yaw']
    contr_cols = ['ctrl_stamp_ns', 'acceleration_level', 'steering']

    # interpolate values between max and min values
    for col in coords_cols:
        control_3m[col] = interpolate_coords(control_3m, f'{col}_min', f'{col}_max')

    # select coords and control columns
    control_interpolated = control_3m[contr_cols + coords_cols]
   
 
    clm = add_metadata(control_interpolated, metadata)

    # add acceleration_level and steering columns with shifts
    for col in ['acceleration_level', 'steering']:
        for i in range(1, 4):
            clm[f'{col}_shift_{i}'] = clm[col].shift(i)

    # add x, y, yaw columns with shifts
    for col in ['x', 'y', 'yaw']:
        for i in range(1, 4):
            clm[f'{col}_shift_{i}'] = clm[col].shift(i)

    # add mean last 10 values for acceleration_level and steering columns
    for col in ['acceleration_level', 'steering']:
        clm[f'{col}_last_10_mean'] = clm[col].rolling(window=10).mean()

    # add mean last 10 values for x, y, yaw columns
    for col in ['x', 'y', 'yaw']:
        clm[f'{col}_last_10_mean'] = clm[col].rolling(window=10).mean()



    return clm

In [20]:
ids = [0]
targets = ['x']

df_list = []
for id in ids:
    control = client.query_df(f'''
        select * 
        from ycup.control yc
        where yc.id = {id}
        limit 10'''
    )

    localizations = client.query_df(f'''
        select * 
        from ycup.localization yl
        where yl.id = {id}
        limit 10'''
    )

    metadata = client.query_df(f'''
        select * 
        from ycup.metadata ym
        where ym.id = {id}
        limit 10'''
    )
    # add df by id to list
    df_list.append(union_dfs(control, localizations, metadata)) 

df = pd.concat(df_list, ignore_index=True)    
                    
df_prepr = {}
for target in targets:
    df_prepr[target] = df_preprocessor(df, target, id, preprocessor_path, targets)

/opt/bitnami/python/lib/python3.12/site-packages/sklearn/base.py:411: FutureWarning: The `_get_tags` method is deprecated in 1.6 and will be removed in 1.7. Please implement the `__sklearn_tags__` method.
  warnings.warn(
/opt/bitnami/python/lib/python3.12/site-packages/sklearn/utils/_tags.py:354: FutureWarning: The CatBoostEncoder or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(


AttributeError: 'super' object has no attribute '__sklearn_tags__'

### spark connect

In [4]:
db = 'ycup'
# list of tables in db
tables = spark.sql(f'SHOW TABLES in {db}').collect()

df_list = {}
for table in tables:
    table_name = table.tableName
    df = spark.sql(f'SELECT * FROM {db}.{table_name} limit 10').toPandas()
    df_list[table_name] = df

In [ ]:
# second variant


In [5]:
df_list['control']

,id,stamp_ns,acceleration_level,steering
0,0,36479492,-929,5.739836
1,0,76459951,-926,5.280618
2,0,116678417,-918,5.039505
3,0,156788958,-908,4.734873
4,0,196857808,-897,4.387096
5,0,236974997,-892,4.014573
6,0,276890721,-892,3.627408
7,0,316915752,-901,3.191926
8,0,356884436,-911,2.748362
9,0,396895329,-918,2.286391


## Client-Side: Sending and Receiving Data

In [13]:
result = requests.post('http://127.0.0.1:8000/get_df', params={'id': 1})
result

ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=8000): Max retries exceeded with url: /get_df?id=1 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f558e563d70>: Failed to establish a new connection: [Errno 111] Connection refused'))

## Running the Server


```
uvicorn your_script_name:app --reload
'''
